# Assignment 1

In [2]:
%config IPCompleter.greedy=True


import pandas as pd
import os
import re

from gensim.similarities import Similarity
from scipy import spatial
from pathlib import Path


from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse
from nltk.stem import PorterStemmer

from sklearn.feature_extraction.text import TfidfVectorizer


ModuleNotFoundError: No module named 'gensim'

In [1]:
training_dir_path = "Training"
training_data = pd.DataFrame(index=None, columns=['current_dir','current_fold','input_reviews','fold','output_class'])

NameError: name 'pd' is not defined

In [29]:
total_data = []

for dir in os.listdir(training_dir_path):
    current_fold = dir
    full_path_of_folds = os.path.join(training_dir_path, dir)
    for dir in os.listdir(full_path_of_folds):
        if dir == "deceptive":
            output_class = 1
            full_path_of_current_deceptive = os.path.join(full_path_of_folds, dir)
            for dir in os.listdir(full_path_of_current_deceptive):
                with open(os.path.join(full_path_of_current_deceptive, dir)) as f:
                    contents = ""
                    for line in f.readlines():
                        contents += line
                    row_dict = {'current_dir':"deceptive" ,'input_reviews':contents, 'current_fold':current_fold, 'output_class':output_class}
                    total_data.append(row_dict)
        
        if dir == "truthful":
            output_class = 0
            full_path_of_current_deceptive = os.path.join(full_path_of_folds, dir)
            for dir in os.listdir(full_path_of_current_deceptive):
                with open(os.path.join(full_path_of_current_deceptive, dir)) as f:
                    contents = ""
                    for line in f.readlines():
                        contents += line
                    row_dict = {'current_dir':"truthful" ,'input_reviews':contents, 'current_fold':current_fold, 'output_class':output_class}
                    total_data.append(row_dict)


training_data = pd.DataFrame(total_data)

In [30]:

# https://gist.github.com/sebleier/554280

stopwords= set(['br', 'the', 'i', 'me', 'my', 'myself', 'we', 'our', 'ours', 'ourselves', 'you', "you're", "you've",\
            "you'll", "you'd", 'your', 'yours', 'yourself', 'yourselves', 'he', 'him', 'his', 'himself', \
            'she', "she's", 'her', 'hers', 'herself', 'it', "it's", 'its', 'itself', 'they', 'them', 'their',\
            'theirs', 'themselves', 'what', 'which', 'who', 'whom', 'this', 'that', "that'll", 'these', 'those', \
            'am', 'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had', 'having', 'do', 'does', \
            'did', 'doing', 'a', 'an', 'the', 'and', 'but', 'if', 'or', 'because', 'as', 'until', 'while', 'of', \
            'at', 'by', 'for', 'with', 'about', 'against', 'between', 'into', 'through', 'during', 'before', 'after',\
            'above', 'below', 'to', 'from', 'up', 'down', 'in', 'out', 'on', 'off', 'over', 'under', 'again', 'further',\
            'then', 'once', 'here', 'there', 'when', 'where', 'why', 'how', 'all', 'any', 'both', 'each', 'few', 'more',\
            'most', 'other', 'some', 'such', 'only', 'own', 'same', 'so', 'than', 'too', 'very', \
            's', 't', 'can', 'will', 'just', 'don', "don't", 'should', "should've", 'now', 'd', 'll', 'm', 'o', 're', \
            've', 'y', 'ain', 'aren', "aren't", 'couldn', "couldn't", 'didn', "didn't", 'doesn', "doesn't", 'hadn',\
            "hadn't", 'hasn', "hasn't", 'haven', "haven't", 'isn', "isn't", 'ma', 'mightn', "mightn't", 'mustn',\
            "mustn't", 'needn', "needn't", 'shan', "shan't", 'shouldn', "shouldn't", 'wasn', "wasn't", 'weren', "weren't", \
            'won', "won't", 'wouldn', "wouldn't"])

def abbreviated(phrase):
    phrase = re.sub(r"won't", "will not", phrase)
    phrase = re.sub(r"can\'t", "can not", phrase)

    # general
    phrase = re.sub(r"n\'t", " not", phrase)
    phrase = re.sub(r"\'re", " are", phrase)
    phrase = re.sub(r"\'s", " is", phrase)
    phrase = re.sub(r"\'d", " would", phrase)
    phrase = re.sub(r"\'ll", " will", phrase)
    phrase = re.sub(r"\'t", " not", phrase)
    phrase = re.sub(r"\'ve", " have", phrase)
    phrase = re.sub(r"\'m", " am", phrase)
    return phrase


def preprocess_text(sentence):
    ps = PorterStemmer()
    preprocessed_txt = re.sub("http\S+", "", sentence)
    preprocessed_txt = abbreviated(preprocessed_txt)
    preprocessed_txt = re.sub("\S*\d\S*", "", preprocessed_txt).strip()
    preprocessed_txt = re.sub('[^A-Za-z]+', ' ', preprocessed_txt)
    preprocessed_txt = re.sub('\\t', ' ', preprocessed_txt)
    preprocessed_txt = ' '.join(e.lower() for e in preprocessed_txt.split() if e.lower() not in stopwords)
    preprocessed_txt = ' '.join(ps.stem(e) for e in preprocessed_txt.split())
    
    return preprocessed_txt.strip()
    
training_data['preprocessed_reviews'] = training_data['input_reviews'].apply(preprocess_text)

training_data.head()

,current_dir,current_fold,input_reviews,output_class,preprocessed_reviews
0,deceptive,Fold1,We stayed at the Schicago Hilton for 4 days an...,1,stay schicago hilton day night confer say norm...
1,deceptive,Fold1,Hotel is located 1/2 mile from the train stati...,1,hotel locat mile train station quit hike trave...
2,deceptive,Fold1,"When most people think Hilton, they think luxu...",1,peopl think hilton think luxuri know wish woul...
3,deceptive,Fold1,My husband and I recently stayed stayed at the...,1,husband recent stay stay hilton chicago horrib...
4,deceptive,Fold1,My wife and I booked a room at the Hilton Chic...,1,wife book room hilton chicago three weekend ag...


In [31]:
# def split_into_words(sentence):    
#     return sentence.split()

# training_data['preprocessed_words'] = training_data['preprocessed_reviews'].apply(split_into_words)

# training_data.head()

In [ ]:
def get_tf_idf_similarity(test_data, train_data, k):
    tf_idf_vect = TfidfVectorizer()
    train_docs = [test_data]
    
    for iter, row in train_data['preprocessed_reviews'].iteritems():
        train_docs.append(row)

    train_matrix = tf_idf_vect.fit_transform(train_docs)


    sim = cosine_similarity(train_matrix[0:1], train_matrix[1:])
    
    array_indices = (sim[0].argsort()[-k:][::-1]);
    
    print('Array indices are : ',array_indices, " and its len is : ", len(array_indices))
    
    neg_review_count = 0
    pos_review_count = 0

    for idx in array_indices:
        if (train_data['output_class'].iloc[array_indices[0]] == 0):
            pos_review_count += 1
        else:
            neg_review_count += 1
    
    if neg_review_count > pos_review_count:
        return 1
    else:
        return 0
            

In [33]:
fold_dir_prefix = "Fold"

training_dir_path = "../Test"
total_data = []


for dir in os.listdir(training_dir_path):
    with open(os.path.join(training_dir_path, dir)) as f:
        contents = ""
        for line in f.readlines():
            contents += line
        row_dict = {'file_name':Path(dir).stem, 'test_text':contents}
        total_data.append(row_dict)
        
test_data = pd.DataFrame(total_data)

# test_data = test_data.set_index('file_name')

test_data.head()


,file_name,test_text
0,1,My husband and I loved this hotel. The staff w...
1,10,Wow! Loved the Ambassador East Hotel in Chicag...
2,100,"I reserved the ""Rock star"" suite for my boyfri..."
3,101,Having come to Chicago with my three sisters f...
4,102,I was highly disappointed with my choice to st...


In [34]:
test_data['preprocessed_reviews'] = test_data['test_text'].apply(preprocess_text)

test_data.head()

,file_name,test_text,preprocessed_reviews
0,1,My husband and I loved this hotel. The staff w...,husband love hotel staff friendli food room se...
1,10,Wow! Loved the Ambassador East Hotel in Chicag...,wow love ambassador east hotel chicago close g...
2,100,"I reserved the ""Rock star"" suite for my boyfri...",reserv rock star suit boyfriend birthday month...
3,101,Having come to Chicago with my three sisters f...,come chicago three sister relax weekend shop s...
4,102,I was highly disappointed with my choice to st...,highli disappoint choic stay amalfi hotel chic...


In [47]:


i = 0
output = []
while i < test_data.shape[0] :
    filtered_df = test_data[test_data['file_name'] == str(i+1)]
    print(filtered_df)
    output.append(get_tf_idf_similarity(filtered_df['preprocessed_reviews'].iloc[0], training_data, 95))
    i+=1

array = np.asarray(output)

np.savetxt('results-tfidf-95.txt', array, fmt='%i')


  file_name                                          test_text  \
0         1  My husband and I loved this hotel. The staff w...   

                                preprocessed_reviews  
0  husband love hotel staff friendli food room se...  
   file_name                                          test_text  \
72         2  After arriving at the Sofitel Chicago Water To...   

                                 preprocessed_reviews  
72  arriv sofitel chicago water tower hotel greet ...  
   file_name                                          test_text  \
83         3  Great hotel! Went to see the Museum of Contemp...   

                                 preprocessed_reviews  
83  great hotel went see museum contemporari art g...  
   file_name                                          test_text  \
94         4  I stayed here with a dozen of my friends, and ...   

                                 preprocessed_reviews  
94  stay dozen friend half us got eaten aliv bed b...  
    file_name   

   file_name                                          test_text  \
91        37  My fiance and I traveled to Chicago for the fi...   

                                 preprocessed_reviews  
91  fianc travel chicago first time decemb not hap...  
   file_name                                          test_text  \
92        38  My stay at the Homewood Suites in downtown Chi...   

                                 preprocessed_reviews  
92  stay homewood suit downtown chicago without do...  
   file_name                                          test_text  \
93        39  My stay at the Omni Chicago Hotel was wonderfu...   

                                 preprocessed_reviews  
93  stay omni chicago hotel wonder last minut busi...  
   file_name                                          test_text  \
95        40  During my recent stay at the Hyatt Regency Chi...   

                                 preprocessed_reviews  
95  recent stay hyatt regenc chicago sever disappo...  
   file_name

    file_name                                          test_text  \
128        70  My husband and I stayed at The Palmer House Hi...   

                                  preprocessed_reviews  
128  husband stay palmer hous hilton last month cel...  
    file_name                                          test_text  \
129        71  I got a Sunday night stay for only $50 off of ...   

                                  preprocessed_reviews  
129  got sunday night stay pricelin com would hard ...  
    file_name                                          test_text  \
130        72  This fantastic hotel has it all: luxury, an ex...   

                                  preprocessed_reviews  
130  fantast hotel luxuri excel locat fun atmospher...  
    file_name                                          test_text  \
131        73  Going to the Internet Retailer 2010 at the las...   

                                  preprocessed_reviews  
131  go internet retail last minut hotel left avail..

  file_name                                          test_text  \
5       103  My husband and I stayed at the Omni Chicago Ho...   

                                preprocessed_reviews  
5  husband stay omni chicago hotel last weekend f...  
  file_name                                          test_text  \
6       104  My husband and I stayed at the Sheraton Chicag...   

                                preprocessed_reviews  
6  husband stay sheraton chicago hotel tower febr...  
  file_name                                          test_text  \
7       105  Stayed for 3 nights. booked via hotwire, upgra...   

                                preprocessed_reviews  
7  stay night book via hotwir upgrad histor tower...  
  file_name                                          test_text  \
8       106  We loved the hotel. When I see other posts abo...   

                                preprocessed_reviews  
8  love hotel see post shabbi not life figur talk...  
  file_name                 

   file_name                                          test_text  \
43       138  I had purchased this deal of off groupon. Lets...   

                                 preprocessed_reviews  
43  purchas deal groupon let say found bunch hidde...  
   file_name                                          test_text  \
44       139  I frequently have business meetings in Downtow...   

                                 preprocessed_reviews  
44  frequent busi meet downtown chicago find hotel...  
   file_name                                          test_text  \
46       140  I was not pleased with my recent stay at the P...   

                                 preprocessed_reviews  
46  not pleas recent stay palmer hous hilton room ...  
   file_name                                          test_text  \
47       141  I accompanied my husband on a business trip to...   

                                 preprocessed_reviews  
47  accompani husband busi trip chicago back septe...  
   file_name

In [263]:
# test_data['predicted_output'] = test_data['preprocessed_reviews'].apply(get_tf_idf_similarity, args = (training_data, 5,))
# np.savetxt('results-tfidf-37-new.txt', (test_data['predicted_output']), fmt='%i')

## Cross validation 

In [49]:
folds = ['Fold1','Fold2','Fold3','Fold4','Fold5','Fold6','Fold7','Fold8','Fold9','Fold10']

In [57]:
for k in range (5,12,4):
    accuracy_for_all_folds = 0
    for fold in folds:
        print('Staring processing for ', fold)
        training_data_with_fold_excluded = training_data[training_data['current_fold'] != fold]
        training_data_in_fold = training_data[training_data['current_fold'] == fold]

        training_data_in_fold['predicted_output'] = training_data_in_fold['preprocessed_reviews'].apply(get_tf_idf_similarity, args = (training_data_with_fold_excluded, k,))
        accurate_predictions = training_data_in_fold[training_data_in_fold['output_class'] == training_data_in_fold['predicted_output'] ].shape[0]

        accuracy_for_current_fold = (accurate_predictions/training_data_in_fold.shape[0])
        print('Accuracy for the ', fold, ' is :', accuracy_for_current_fold)

        accuracy_for_all_folds += accuracy_for_current_fold

    print('For k : ', k ,'Avg accuracy for 10 folds is : ', (accuracy_for_all_folds/10))
    print("="*50)


Staring processing for  Fold1
Array indices are :  [ 337  559  222  195 1165]
Array indices are :  [531 388 164 553 406]
Array indices are :  [172 614 613 640 199]
Array indices are :  [559 808 358 495 476]
Array indices are :  [715 513 338 453 661]
Array indices are :  [707 447 240 191 310]
Array indices are :  [640 305 231 213 165]
Array indices are :  [347 436 325 474 495]
Array indices are :  [423 605 299 315 144]
Array indices are :  [1041  447 1129  310  740]
Array indices are :  [736  67 724  58 161]
Array indices are :  [317 406 355 213 804]
Array indices are :  [ 451  592 1173  726  604]
Array indices are :  [156 625 472 642 531]
Array indices are :  [447 658 349 302 471]
Array indices are :  [347 517 325 673 495]
Array indices are :  [500   8 442 213 613]
Array indices are :  [715 270 148 330 153]
Array indices are :  [513 625 455 301 164]
Array indices are :  [ 254  199 1139  618  589]
Array indices are :  [165 586 653 531 330]
Array indices are :  [749 750 741 199 756]
Arra

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


Array indices are :  [949 451 602 435 356]
Array indices are :  [ 864  869  939  942 1281]
Array indices are :  [714 672 328 600 345]
Array indices are :  [  32   49 1277  337 1140]
Array indices are :  [375 339 307 612 404]
Array indices are :  [133  49 927 643 564]
Array indices are :  [864 618 820 878 871]
Array indices are :  [485  11 583 864  17]
Array indices are :  [864 868 214 869   1]
Array indices are :  [1171  864  914 1067   39]
Array indices are :  [ 13 556 359 971 303]
Array indices are :  [487 602 292 864 329]
Array indices are :  [ 476  864  530 1268  470]
Array indices are :  [115 873 411 870 524]
Array indices are :  [864 563 872 590 869]
Array indices are :  [ 630   11 1129  643  528]
Array indices are :  [969 892 882 899 683]
Array indices are :  [  84 1167  955  898   13]
Array indices are :  [884 405 512 886 387]
Array indices are :  [1137  512  454  501  652]
Array indices are :  [367 296 475 886 341]
Array indices are :  [ 496  531 1207  405  624]
Array indices 

Array indices are :  [339  48  17  18 345]
Array indices are :  [469 476 474 436 292]
Array indices are :  [ 625  517  357  523 1049]
Array indices are :  [583 655 604 601 341]
Array indices are :  [477 454  11 325 589]
Array indices are :  [ 594 1052  451  394 1056]
Array indices are :  [1048 1056 1046 1049 1062]
Array indices are :  [1048 1056 1049 1053 1046]
Array indices are :  [1049  304  445  310  292]
Array indices are :  [600 133  82   0 549]
Array indices are :  [513   4 644 869 341]
Array indices are :  [1123  559  708  466 1129]
Array indices are :  [ 71 400 294 447  13]
Array indices are :  [665 357 829 690 366]
Array indices are :  [485 109 310 715 292]
Array indices are :  [447 580 528 564 540]
Array indices are :  [ 69 329 602  73 172]
Array indices are :  [ 343 1076 1078 1071 1067]
Array indices are :  [673 357 292 517  15]
Array indices are :  [517  83 474 673  15]
Array indices are :  [527 497  15 454  63]
Array indices are :  [496  14  76 636 444]
Array indices are :

Array indices are :  [ 373 1117  376  378 1142]
Array indices are :  [998  40 603  84  49]
Array indices are :  [378  26 527 587 586]
Array indices are :  [1269  289  244  571  642]
Array indices are :  [ 547  677  414   70 1254]
Array indices are :  [ 26   4  18 482 309]
Array indices are :  [676 124 666 418  98]
Array indices are :  [515 625 316 651 845]
Array indices are :  [107 398 136 350 580]
Array indices are :  [673 469 458 470 491]
Array indices are :  [ 677   66  124 1263  414]
Array indices are :  [1236  779  632  667 1022]
Array indices are :  [107 139 398  15  11]
Array indices are :  [469 673 470 357 473]
Array indices are :  [ 315  527 1139  673  310]
Array indices are :  [673 414 468 345 451]
Array indices are :  [ 484  387  345  632 1187]
Array indices are :  [1129 1236  857  825  550]
Array indices are :  [1271  268 1123  526  151]
Array indices are :  [416 357 313 673 492]
Array indices are :  [ 86 487 625 633 661]
Array indices are :  [718  11 973  80 305]
Array ind

Array indices are :  [605 316 566 197  63]
Array indices are :  [263 698  76 789 424]
Array indices are :  [1069 1067  193  254  780]
Array indices are :  [ 76 713 488 614  15]
Array indices are :  [  4 552 420 416 418]
Array indices are :  [670 560 563 829 543]
Array indices are :  [ 605  309 1168  550  832]
Array indices are :  [151 197 162 190 181]
Array indices are :  [1162  812  441 1081 1019]
Array indices are :  [300 410 598 530 104]
Array indices are :  [1117  474  552  406   15]
Array indices are :  [ 280  689 1168 1165  334]
Array indices are :  [211 203 133 204 277]
Array indices are :  [ 552  109  437  658 1129]
Array indices are :  [390 657 550 616 357]
Array indices are :  [ 281 1218  887  188  151]
Array indices are :  [335   5 993 164 538]
Array indices are :  [445   4 657 483 656]
Array indices are :  [278 539 543 709  48]
Array indices are :  [  13   48 1129   71  520]
Array indices are :  [1031   88  274  773  123]
Array indices are :  [449  13  84 376 309]
Array ind

KeyboardInterrupt: 